# Normal Day Analysis

In [2]:
import pandas as pd
import numpy as np
import ipaddress
import dns.resolver
import dns.reversename
import pygeoip
import matplotlib.pyplot as plt 

In [3]:
datafile='data7.parquet'

In [4]:
gi=pygeoip.GeoIP('./GeoIP.dat')
gi2=pygeoip.GeoIP('./GeoIPASNum.dat')
addr='193.136.73.21'
cc=gi.country_code_by_addr(addr)
org=gi2.org_by_addr(addr)
print(cc,org)

PT AS1930 Fundacao para a Ciencia e a Tecnologia, I.P.


In [5]:
data=pd.read_parquet(datafile)

In [6]:
c.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
709948,1232265,192.168.107.196,157.240.212.35,tcp,443,13896,113280
709949,1232398,192.168.107.196,157.240.212.35,tcp,443,45276,419472


In [ ]:
NET=ipaddress.IPv4Network('192.168.107.0/24')
private_dst_normal = data[data.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) in NET,axis=1)]

private_dst_normal.head()


In [ ]:
grp_private_normal = private_dst_normal.groupby(["proto","port"])['dst_ip'].apply(list)

In [ ]:
print(set(grp_private_normal[('tcp', 443)]))
print(set(grp_private_normal[('udp', 53)]))

## Servers 
tcp 443 {'192.168.107.223', '192.168.107.238', '192.168.107.222', '192.168.107.226'} https

udp 53 {'192.168.107.227', '192.168.107.234'} dns
